# Getting Started with the Parameter Sweep Tool

Parameter sweep is one way of performing optimization, sensitivity analysis, and uncertainty quantification where experiments are run repeatedly with a different set of input parameters to see their effects on the output qunatities of interest. Output metrics can include levelized cost of water (LCOW), component cost, specific energy consumption, energy efficiency, water recovery rate, gained output ratio, and bulk temperature difference among others. Input parameters are technology dependent, some of which include feed water salinity, membrane permeability, membrane area, pumping efficiency, thermal conductivity, operating temperature, number of stages, labor, and capital expenditure.

This notebook will demonstrate how the parameter sweep tool in WaterTAP can be used for various technoeconomic analyses.

## Types of Parameter Sweeps

In its current form, a user can run 3 types of parameter sweeps

* **Simple Parameter Sweep** : Generate samples from a distribution or an Euclidean space and solve flowsheets with those input values.
* **Recursive Parameter Sweep** : Will run the simple parameter sweep in recursion if some runs fail to ensure that a user-specified number of sample results are generated. This involves resampling the input parameter space to compensate for the failed runs.
* **Differential Parameter Sweep** : This runs a differential sweep to gather sensitivity data when the input parameter space is more than one dimensional. A differential sweep comprises of two types of sweeps, an outer "nominal" sweep and an inner "differential" sweep. The nominal sweep is a simple parameter sweep over the sampled space. The differential sweep is a simple parameter sweep that occurs at every nominal value, where one of the sweep paramters is preturbed keeping the others fixed to their nominal values.

## Flowsheet Example

We use a simple RO system with an energy recovery device to demonstrate the capabilities of the parameter sweep tool. It comprises of an inlet high pressure pump connected to a steady state zero-dimensional RO process model.
This RO model is designed to treat a single liquid phase model. A turbine-type isothermal energy recovery device model is connected downstream of the RO model. The feed water is an NaCl brine solution and is modeled using an property package within WaterTAP. Outputs that can be measured from this flow sheet include LCOW, product flow rate and concentration, volumetric recovery, water recovery, and specific energy consumption.

<img src="parameter_sweep_demo/RO_ERD_flowsheet.png" alt="RO with energy recovery device" width="400"/>

In [ ]:
# Make the necessary imports
from pprint import pprint
from IPython import get_ipython
from idaes.core.solvers import get_solver
from watertap.examples.flowsheets.RO_with_energy_recovery.RO_with_energy_recovery import (
    build,
    set_operating_conditions,
    initialize_system,
    optimize,
)
from watertap.examples.flowsheets.RO_with_energy_recovery.monte_carlo_sampling_RO_ERD import (
    get_sweep_params,
    build_model,
    build_sweep_params,
    build_outputs,
    run_parameter_sweep,
)



In [ ]:
??get_sweep_params

In [ ]:
??build_model

In [ ]:
??build_sweep_params

In [ ]:
??build_outputs

In [ ]:
from watertap.tools.parameter_sweep import (
    UniformSample,
    NormalSample,
    LatinHypercubeSample,
    ParameterSweep,
)

In [ ]:
def run_parameter_sweep(num_samples=100, num_procs=1):
    
    # solver = get_solver()
    ps = create_parameter_sweep_object(solver, num_samples, num_procs)
    results_dict, results_array = ps.parameter_sweep(
        build_model,
        build_sweep_params,
        build_outputs=None,
        build_outputs_kwargs=None,
        num_samples=None,
        seed=None,
        build_model_kwargs=None,
        build_sweep_params_kwargs=None,
    )

In [ ]:
def create_parameter_sweep_object(num_samples, num_procs):
    # pprint(ParameterSweep.CONFIG.display())
    
    solver = get_solver()
    kwargs_dict = {
        "debugging_data_dir": None,
        "csv_results_file_name" : None,
        "h5_results_file_name" : None,
        "interpolate_nan_outputs" : False,
        "h5_parent_group_name" : None,
        "build_model" : build_model,
        "build_model_kwargs" : dict(
                                read_model_defauls_from_file=False,
                                defaults_fname="default_configuration.yaml",
                            ),
        "build_sweep_params" : build_sweep_params,
        "build_sweep_params_kwargs" : dict(
                                        num_samples=num_samples,
                                        use_LHS=False,
                                        sweep_params_fname="mc_sweep_params.yaml",
                                        read_sweep_params_from_file=False,
                                    ),
        "build_outputs" : build_outputs,
        "build_outputs_kwargs" : {},
        "optimize_function" : optimize,
        "optimize_kwargs" : {"solver": solver, "check_termination": False},
        "initialize_function" : None,
        "update_sweep_params_before_init" : False,
        "initialize_kwargs" : {},
        "initialize_before_sweep" : False,
        "reinitialize_function" : None,
        "reinitialize_kwargs" : {},
        "reinitialize_before_sweep" : False,
        "probe_function" : None,
        "custom_do_param_sweep" : None,
        "custom_do_param_sweep_kwargs" : {},
        "publish_progress" : False,
        "publish_address" : "http://localhost:8888",
        "number_of_subprocesses" : num_procs,
        "parallel_back_end" : "ConcurrentFutures", # "MultiProcessing",
        "log_model_states" : False,
    }
    ps = ParameterSweep(**kwargs_dict)
    return ps, kwargs_dict

In [ ]:
num_samples = 100
num_procs = 8
ps, kwargs_dict = create_parameter_sweep_object(num_samples, num_procs)

In [ ]:
results_array, results_dict = ps.parameter_sweep(
    kwargs_dict["build_model"],
    kwargs_dict["build_sweep_params"],
    build_outputs = kwargs_dict["build_outputs"],
    build_outputs_kwargs = kwargs_dict["build_outputs_kwargs"],
    num_samples = num_samples,
    seed=None,
    build_model_kwargs = kwargs_dict["build_model_kwargs"],
    build_sweep_params_kwargs = kwargs_dict["build_sweep_params_kwargs"]
)

In [ ]:
pprint(results_dict)

In [ ]:
pprint(results_array)

## Plotting Results

The resulting H5 files can then be prcessed to generate plots, e.g., 2D scatter plots, map plots, etc.

<table><tr>
    <td> <img src="parameter_sweep_demo/scatter_LCOW_vs_Acomp.jpg" alt="LCOW vs water permeability" width="100%"/> </td>
    <td> <img src="parameter_sweep_demo/maps_LCOW_recovery_NaCl_loading.jpg" alt="LCOW vs water recovery and NaCL concentration" width="100%"/> </td>
</tr></table>


# Advanced Usage

## Parallel Manager

The parallel manager allows the end-user to select which parallel backend to run their parameter sweep with. Currently supported parallel backends include:

* Message Passing Interface (MPI)
* Python multiprocessing
* Python concurrent futures
* Ray Core
* Serial

The parallel manager provides a unified API to use the parallel backends.

### Motivation

* Most people will be running parameter sweep on a shared memory system and are familiar with python multiprocessing and concurrent futures.
* MPI enables distributed parallel computing on an HPC
* Ray is an MPI alternative for distributed parallel computing that has a simpler API.

Multiple parallel backends necessitates an abstraction layer such that the onus of supporting these parallel paradigms does not fall on the average WaterTAP developer. Parallel Manager is that abstraction layer.

### Currently Supported Parallel Features

* Gather, all gather - Gather value on a specific or all process(es)
* Scatter - Scatter values from one process to all other processess
* Broadcast - Broadcast a data structure to all processes
* All reduce - Reduce values and distribute to all processes

### Benchmarking Results

Next we look at benchmarking results of running the parameter sweep tool

#### Python Multiprocessing

#### MPI on Eagle HPC

## Loop Tool